In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('labeled_10k.csv') 
print("Dataset Loaded : Done")
dataset

Dataset Loaded : Done


,hmid,moment,concepts,agency,social,age,country,gender,married,parenthood,reflection,duration
0,27674,I was happy when my son got 90% marks in his e...,education|family,no,yes,29.0,IND,m,married,y,24h,half_a_day
1,27685,went to movies with my friends it was fun,entertainment,yes,yes,29.0,IND,m,single,y,24h,half_a_day
2,27691,A hot kiss with my girl friend last night made...,romance,yes,yes,25.0,IND,m,married,y,24h,at_least_one_hour
3,27701,My son woke me up to a fantastic breakfast of ...,family|food,no,yes,79,USA,f,widowed,y,24h,all_day_im_still_feeling_it
4,27712,My older daughter keeps patting my younger dau...,family,no,yes,30,USA,f,married,y,24h,a_few_moment
...,...,...,...,...,...,...,...,...,...,...,...,...
10555,128748,My husband called me just to tell me he loved me.,romance|family,no,yes,58,USA,f,married,n,24h,a_few_minutes
10556,128753,"I worked out, which always makes me feel good.",career,yes,no,24,USA,m,married,y,24h,at_least_one_hour
10557,128754,Finally got to watch the new Resident Evil movie.,entertainment,yes,no,34,USA,m,divorced,y,24h,at_least_one_hour
10558,128757,I got to talk to an old friend and reminisce o...,conversation,yes,yes,28,USA,f,single,y,24h,at_least_one_hour


In [3]:
from sklearn.preprocessing import LabelEncoder

df = dataset
dummies = pd.get_dummies(df.country)
merged = pd.concat([df,dummies], axis='columns')

print("Data Processing 1 : Done")

Data Processing 1 : Done


In [4]:
final = merged.drop(['country', 'SWE'],axis='columns')

Y_agency = final.agency
Y_social = final.social


X = final.drop('agency',axis='columns')
X = X.drop('social',axis='columns')

dummies = pd.get_dummies(X.concepts)
X = pd.concat([X,dummies], axis='columns')

X = X.drop(['gender', 'hmid', 'concepts', 'animals', 'married', 'parenthood', 'reflection', 'duration'],axis='columns')

print("Data Processing 2 : Done")

Data Processing 2 : Done


In [5]:
#X["age"] = apply(pd.to_numeric(X["age"]))

X[["age"]] = X[["age"]].apply(pd.to_numeric,  errors='coerce')

print("Data Processing 3 : Done")

Data Processing 3 : Done


In [6]:
tmp = X
tmp = tmp.drop(['moment'],axis='columns')

print("Data Processing 4 : Done")

Data Processing 4 : Done


In [7]:
#df1 = dataset[['moment']]
import nltk
from nltk import ngrams
from collections import defaultdict 

moment_List = dataset['moment'].values
data = []
data_process = []
count_data = defaultdict(int)

for count,value in enumerate(moment_List):
    Four_grams = ngrams(value.split(), 4)
    token_freq = nltk.FreqDist(Four_grams)
    for token in token_freq.keys():
        str1=""
        for val in token:
            str1 += val+" "
        data_process.append(str1)
        data.append((count,str1))
        count_data[count] += 1

print("Prepare Anagram List : Done")

Prepare Anagram List : Done


In [8]:
agency      = dataset['agency'].values
social      = dataset['social'].values

for n, i in enumerate(agency):
    if i == 'no':
        agency[n]=0
    else:
        agency[n]=1
        
for n, i in enumerate(social):
    if i == 'no':
        social[n]=0
    else:
        social[n]=1
        
print("Agency Value set : Done")
print("Social Value set : Done")

Agency Value set : Done
Social Value set : Done


In [9]:
column_values = pd.Series(data_process)
df = pd.DataFrame(data) 

y_agency=[]
for i,j in zip(count_data.values(), agency):
    for val in range(0,i):
        y_agency.append(j)
        
y_social=[]
for i,j in zip(count_data.values(), social):
    for val in range(0,i):
        y_social.append(j)

print("Y_Agency Prepared : Done")
print("Y_Social Prepared : Done")


Y_Agency Prepared : Done
Y_Social Prepared : Done


In [10]:
def split_list(a_list, y):
    return a_list[:y], a_list[y+1:]


x = int(0.2*len(y_agency))+1
y = int(len(y_agency) - 0.2*len(y_agency))
y_agency_train, y_agency_test = split_list(y_agency,y)

x_train_, x_test_ = split_list(data_process, y)
y_social_train, y_social_test = split_list(y_social, y)

print("done")

done


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(min_df=1, stop_words='english')
x_train = tf_idf.fit_transform(x_train_)
x_test = tf_idf.transform(x_test_)

x_train.toarray()
print(x_train.shape)

print("Vectorizer : Done")



(88187, 6422)
Vectorizer : Done


In [13]:
# Predicting Social

from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(activation='logistic', solver='sgd', hidden_layer_sizes=(100,50), random_state = 1)

# model = LogisticRegression(solver='liblinear')
# y_agency=y_agency.astype('int')

nn.fit(x_train, y_social_train)

y_pred = nn.predict(x_test)

count=0
for i in range(len(y_pred)):
    if y_pred[i] == y_social_test[i]:
        count += 1

print("Neural Network Social : ",count/len(y_pred) * 100,"% accuracy")

print("Neural Network Model Fit : Done")


Neural Network Social :  49.19713326680577 % accuracy
Neural Network Model Fit : Done


In [14]:
# Predicting Agency with Activation = Relu

from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(activation='relu', solver='sgd', hidden_layer_sizes=(100,50), random_state = 1)

# model = LogisticRegression(solver='liblinear')
# y_agency=y_agency.astype('int')

nn.fit(x_train, y_agency_train)

y_pred = nn.predict(x_test)

count=0
for i in range(len(y_pred)):
    if y_pred[i] == y_agency_test[i]:
        count += 1

print("Neural Network Agency : ",count/len(y_pred) * 100,"% accuracy")

print("Neural Network Model Fit : Done")


Neural Network Agency :  74.61217454413499 % accuracy
Neural Network Model Fit : Done
